# Review

## Equilibrium

An equilibrium of the differential equation $x'=f(x)$ is a constant solution $x(t)=x^*$, which must satisfy
$$f(x^*)=0$$

## Numerical evaluation
* Use the `subs` method to substitute a float value to a symbol
  * We need to convert a sympy constant **v** to a float point value using `float(v)`

* `lambdify(vars, e)` creates a lambda function from an expression
  * vars are the arugments, and e is the expression

# Stability

Like the equilibrium of a difference equation, the stability of an equilibrium of a differential equation determines if the system remains close to the equilibrium after a small perturbation.
* Stable: remains close
* Asymptotically stable: stable, and the solution after a small perturbation eventually approaches the equilibrium
* Unstable: the solution eventually moves away from the equilibrium

## Theorem:
Consider a system of differential equations $x'=f(x)$, $x(t)=x^*$ is an equilibrium, i.e., $f(x^*)=0$. Here $f(x)$ is continuously differentiable. $J=Df(x^*)$ is the Jacobian matrix of $f(x)$ evaluated at $x^*$, i.e.,
$$
J=\left[\begin{array}{cccc}
\frac{\partial f_1}{\partial x_1} & \frac{\partial f_1}{\partial x_2} &\cdots & \frac{\partial f_1}{\partial x_n}\\
\frac{\partial f_2}{\partial x_1} & \frac{\partial f_2}{\partial x_2} &\cdots & \frac{\partial f_2}{\partial x_n}\\
\vdots & \vdots & \ddots & \vdots\\
\frac{\partial f_n}{\partial x_1} & \frac{\partial f_n}{\partial x_2} &\cdots & \frac{\partial f_n}{\partial x_n}
\end{array}
\right]_{x=x^*}
$$
Then, 
* $x^*$ is asymptotically stable if all eigenvalues of $J$ have negative real parts.
* $x^*$ is unstable if one eigenvalue of $J$ has a positive real part.

## Example: the SIRS model revisited

\begin{align*}
S'&=-\beta SI +\rho (1-S-I)\\
I'&=\beta SI-\gamma I\\
\end{align*}


In [4]:
from sympy import *
from sympy.abc import S, I
beta, gamma, rho = symbols("beta, gamma, rho", positive=True)

dS = -beta*S*I+rho*(1-S-I)
dI = beta*S*I-gamma*I

eq=solve([dS, dI], [S, I])
print(eq)

eqS = simplify(eq[1][0])
eqI = eq[1][1]
display(eq[0], eqS, eqI)

[(1, 0), (-(-rho + rho*(beta - gamma)/beta)/rho, rho*(beta - gamma)/(beta*(gamma + rho)))]


(1, 0)

gamma/beta

rho*(beta - gamma)/(beta*(gamma + rho))

We have found the poositive equilibrium, stored in **(eqS, eqI)**. We can use symbolic differentiation to calculate the Jacobian matrix

### Matrix
The **Matrix** class represents a matrix. Like the construction of a matrix in numpy using array, it takes an array of arrays (each is a row).

In [8]:
Df = Matrix([[diff(dS, S), diff(dS, I)], [diff(dI, S), diff(dI, I)]])
Df

Matrix([
[-I*beta - rho,  -S*beta - rho],
[       I*beta, S*beta - gamma]])

In [10]:
# Evaluate at the equilibrium
J=simplify(Df.subs({S:eqS, I:eqI}))
J

Matrix([
[ -rho*(beta + rho)/(gamma + rho), -gamma - rho],
[rho*(beta - gamma)/(gamma + rho),            0]])

## The eigenvalus

The **.eigenvals** method of a Matrix object calculates the eigenvalues sympolically.
* It takes a no arguments
* It returns a dict of eigenvalues and their multiplicity

In [ ]:
E=J.eigenvals()
E

In [ ]:
es = list(E.keys())
display(es[0], es[1])

It is not straightforward to to determine the sign of the real parts. The following theorem may help.

## Theorem
All the eigenvalues of a $2\times2$ matrix $J$ have negative real parts if and only if $trace(J)<0$ and $det(J)>0$.

In [ ]:
J.trace()

In [ ]:
simplify(J.det())

Thus, the second equilibrium is asymptotically stable if and only if $\beta>\gamma$, which is the same condition as the equilibrim is positive.

# Numeric solutions

Not every initial value problem $x'=f(t, x)$, $x(t_0)=x_0$, can be solved symbolically. But they may have solutions. We may numerically approximate the solution.

## solve_ivp function in the scipy.integrate package
The `solve_ivp(f, t_span, y0)` function in scipy.integrate packge solves initial value problems in the form of
$$\frac{dy}{dt}=f(t, y), y(t_0)=y_0$$
where $t_0$ is `t_span[0]`
* `t_span`: a tuple or a list of two numbers giving the start and end of the time interval on which to find the solution.
  * an optional keyword argument `t_eval`: an array of time points where we want $y(t)$ to be approximated.
* The vector field $f(t, y)$ may have parameters $p$, i.e., $f(t, y, p)$
  * The values of $p$ may be provided by the optional `args` keyword argument


**Return value**: The function returns an object with at least the following instance variables

  * **t**:, an array of time points where the solution is approximated
  * **y**: a matrix of the values of the solution $y(t)$ at the time points.
    * each row corresponds to a variable
    * each column corresponds to a time point
  * **status** an integer encoding the reason for algorithm termination:
    * -1: Integration step failed.
    * 0: The solver successfully reached the end of tspan.
    * 1: A termination event occurred.
  * **message**: a string holding a human-readable description of the termination reason.
  * **success**: a boolean value, True if the solver reached the interval end or a termination event occurred (status >= 0).


# Example The SIRS model revisited

The SIRS model is a two dimensional problem, thus the initial condition is a length-2 array, and $f$ must return a length-2 array.



In [ ]:
beta = 0.4
gamma = 0.2
rho = 1/90

def SIR(t, x):
    # x = [S, I]
    S, I = x
    # returns dS/st, dI/dt
    dS = -beta*S*I + rho*(1-S-I)
    dI = beta*S*I - gamma*I
    return dS, dI

from scipy.integrate import solve_ivp

t_span = [0, 400]
# initial conditions
I0 = 0.0001
S0 = 1-I0
y = solve_ivp(SIR, t_span, [S0, I0])
print(y)

In [ ]:
from matplotlib.pyplot import plot
plot(y.t, y.y[0], "-g", y.t, y.y[1], "-r")

# Example 2: The pendulum
Consider a pendulum:

$$\ddot x +\frac{g}{\ell}\sin x = 0$$

where $g$ is the gravitational constant, $l$ is the length of the rod, and $x$ is the angle between the rod and the vertical condition

This is a second order equation, there is no closed form for the solution, so cannot be solved by sympy.dsolve.

To use **solve_ivp**, we need to convert it into a system of first order equations.

Let $\dot x = y$

\begin{align*}
\dot x &= y\\
\dot y & = \ddot x =-\frac{g}{l}\sin x
\end{align*}

Assume $\ell=2$, $x_0=\pi/6$, $y_0=0$

In [ ]:
from scipy.constants import g
from numpy import sin
from math import pi

#l is an extra parameter
def pendulum(t, xy, l):
    x, y = xy
    dx = y
    dy = -g/l*sin(x)
    return dx, dy

l = 2
x = solve_ivp(pendulum, [0, 10], y0=[pi/6, 0], 
              t_eval = linspace(0, 10, 101),
              # here the args gives the parameters l.
              args=[l])

plot(x.t, x.y[0], '-b', x.t, x.y[1], 'r')

## Make an animation for the pendulum

In [ ]:
from matplotlib.animation import FuncAnimation
from matplotlib.pyplot import subplots

fig, ax = subplots()
p = ax.plot([0, l*sin(x.y[0, 0])], [0, -l*cos(x.y[0,0])], '-ok')
ax.set_aspect(1.0)
ax.set_xlim([-1.1,1.1])
ax.set_ylim([-2.1, 0.1])

def update(i):
    p[0].set_data([0, l*sin(x.y[0, i])], [0, -l*cos(x.y[0,i])])

anim = FuncAnimation(fig, update, frames=range(0, len(x.t)))


In [ ]:
from IPython.display import HTML
HTML(anim.to_html5_video())